In [1]:
from pathlib import Path
from collections import namedtuple
import pandas as pd

In [2]:
logs = Path("/home/jeroen/Documents/Studie/Capita/logs/csv_only/final")
categories = ['Full', 'Wrist', 'Shoulder', 'Humerus']
models = ['single', 'dd-mean', 'dd-mean-std', 'dd-mean-tanh', 'dd-mean-std-tanh', 'fusion-mean', 'fusion-mean-post']
folds = list(range(7))

Metric = namedtuple('Metric', 'col str')

kappa = Metric('cohen-kappa', "Kappa")
loss = Metric('loss', "Loss")

In [3]:
def get_metric_df(metrics):
    df = None
    for category in categories:
        for model in models:
            for fold in [f'fold_{f}' for f in folds]:
                csv = logs / category / model / fold / 'test_score.csv'
                
                if csv.exists():
                    d = pd.read_csv(csv)
                    d = d[[m.col for m in metrics]]

                    d['category'] = category
                    d['model'] = model
                    d['fold'] = fold

                    if df is None:
                        df = d
                    else:
                        df = pd.concat([df, d])
    return df

get_metric_df([kappa, loss])

,cohen-kappa,loss,category,model,fold
0,0.595150,0.453686,Full,single,fold_0
0,0.609700,0.445593,Full,single,fold_1
0,0.586646,0.466130,Full,single,fold_2
0,0.598440,0.456542,Full,single,fold_3
0,0.582775,0.468659,Full,single,fold_4
...,...,...,...,...,...
0,0.644347,0.514034,Humerus,fusion-mean-post,fold_2
0,0.570111,0.465031,Humerus,fusion-mean-post,fold_3
0,0.599934,0.442288,Humerus,fusion-mean-post,fold_4
0,0.673985,0.400458,Humerus,fusion-mean-post,fold_5


In [4]:
def to_report(metrics, math_mode=False):
    formatter = lambda x: f"{x:.3f}"
    
    df = get_metric_df(metrics)
    
    aggregations = {
        f'{metric.col}_{agg}': (metric.col, agg)
        for metric in metrics
        for agg in ['mean', 'std']
    }
    df = df.groupby(['category', 'model']).agg(**aggregations)
    
    for metric in metrics:
        marker = "$" if math_mode else ""
        df[metric.str] = marker + df[f'{metric.col}_mean'].apply(formatter) + "(+-" + df[f'{metric.col}_std'].apply(formatter) + ")" + marker
    
    df = df.reset_index()[['model', 'category'] + [m.str for m in metrics]].melt(id_vars=['model', 'category'], value_vars=[m.str for m in metrics], var_name='metric')

    df = df.pivot(index='model', columns=['category', 'metric'], values='value')
    
    col_order = [
        (c, m.str)
        for c in categories
        for m in metrics
    ]
    df = df[col_order]
    
    row_order = ['single', 'fusion-mean', 'fusion-mean-post', 'dd-mean', 'dd-mean-tanh', 'dd-mean-std', 'dd-mean-std-tanh']
    return df.reindex(row_order)
    
#     scores['metric'] = scores['metric'].apply(formatter) + " (+-" + scores['metric_std'].apply(formatter) + ")"
#     scores[repr_metric] = scores['metric']
#     scores = scores[[repr_metric]]
    
    
#     order = ['single', 'fusion-mean', 'fusion-mean-post', 'dd-mean', 'dd-mean-tanh', 'dd-mean-std', 'dd-mean-std-tanh']
#     res = scores.reset_index().pivot(index='model', columns='category', values=[repr_metric]).reindex(order)
    
#     return res

res = to_report([kappa, loss])
res

category                    Full                           Wrist  \
metric                     Kappa            Loss           Kappa   
model                                                              
single            0.593(+-0.009)  0.458(+-0.009)  0.675(+-0.023)   
fusion-mean       0.564(+-0.006)  0.481(+-0.004)  0.643(+-0.034)   
fusion-mean-post  0.544(+-0.012)  0.500(+-0.009)  0.619(+-0.023)   
dd-mean           0.574(+-0.021)  0.459(+-0.008)  0.676(+-0.032)   
dd-mean-tanh      0.576(+-0.014)  0.460(+-0.010)  0.685(+-0.016)   
dd-mean-std       0.581(+-0.014)  0.455(+-0.008)  0.692(+-0.016)   
dd-mean-std-tanh  0.590(+-0.014)  0.449(+-0.007)  0.670(+-0.026)   

category                                Shoulder                  \
metric                      Loss           Kappa            Loss   
model                                                              
single            0.378(+-0.020)  0.537(+-0.035)  0.512(+-0.020)   
fusion-mean       0.399(+-0.014)  0.482(+-0.039)  0.567(+-0.017)   
fusion-mean-post  0.419(+-0.012)  0.510(+-0.033)  0.555(+-0.030)   
dd-mean           0.361(+-0.015)  0.526(+-0.031)  0.508(+-0.015)   
dd-mean-tanh      0.362(+-0.013)  0.538(+-0.044)  0.518(+-0.026)   
dd-mean-std       0.359(+-0.012)  0.529(+-0.052)  0.517(+-0.011)   
dd-mean-std-tanh  0.356(+-0.013)  0.514(+-0.021)  0.518(+-0.020)   

category                 Humerus                  
metric                     Kappa            Loss  
model                                             
single            0.634(+-0.025)  0.431(+-0.025)  
fusion-mean       0.653(+-0.037)  0.431(+-0.030)  
fusion-mean-post  0.625(+-0.044)  0.454(+-0.034)  
dd-mean           0.755(+-0.027)  0.401(+-0.026)  
dd-mean-tanh      0.754(+-0.024)  0.416(+-0.017)  
dd-mean-std       0.738(+-0.045)  0.392(+-0.017)  
dd-mean-std-tanh  0.727(+-0.029)  0.398(+-0.013)

In [5]:
latex = to_report([kappa, loss], math_mode=False) \
    .to_latex(escape=False) \
    .replace('lllllllll', 'l|ll|ll|ll|ll') \
    .replace('category', '') \
    .replace('metric', '') \
    .replace('model', 'Model') \
    .replace('name', 'Architecture') \
    .replace('+-', '$\pm$')
print(latex)

\begin{tabular}{l|ll|ll|ll|ll}
\toprule
 & \multicolumn{2}{l}{Full} & \multicolumn{2}{l}{Wrist} & \multicolumn{2}{l}{Shoulder} & \multicolumn{2}{l}{Humerus} \\
 &           Kappa &            Loss &           Kappa &            Loss &           Kappa &            Loss &           Kappa &            Loss \\
Model            &                 &                 &                 &                 &                 &                 &                 &                 \\
\midrule
single           &  0.593($\pm$0.009) &  0.458($\pm$0.009) &  0.675($\pm$0.023) &  0.378($\pm$0.020) &  0.537($\pm$0.035) &  0.512($\pm$0.020) &  0.634($\pm$0.025) &  0.431($\pm$0.025) \\
fusion-mean      &  0.564($\pm$0.006) &  0.481($\pm$0.004) &  0.643($\pm$0.034) &  0.399($\pm$0.014) &  0.482($\pm$0.039) &  0.567($\pm$0.017) &  0.653($\pm$0.037) &  0.431($\pm$0.030) \\
fusion-mean-post &  0.544($\pm$0.012) &  0.500($\pm$0.009) &  0.619($\pm$0.023) &  0.419($\pm$0.012) &  0.510($\pm$0.033) &  0.555($\pm$0.030) 